# Reinforcement Learning Modell 

In [28]:
import socket
import json
import time
import torch
import torch.nn as nn
import torch.nn.functional as F

# ===============================
#  MODEL DEFINITION
# ===============================

class TankController(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(4, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 2)

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        return x

# Godot Connector

In [10]:
import socket
import json
from typing import Any, Dict

HOST = "127.0.0.1"
PORT = 5000


def send_json(sock: socket.socket, payload: Dict[str, Any]) -> None:
    """
    Send a single JSON object over the socket.
    """
    data = json.dumps(payload)
    sock.sendall(data.encode("utf-8"))


def on_message(sock: socket.socket, message: Dict[str, Any]) -> None:
    """
    Called every time a JSON message arrives from Godot.

    Here you:
      1) Parse the incoming arena state
      2) Run a training / backprop step on your model
      3) Compute an action/value
      4) Send the result back to Godot via send_json()
    """

    print("\nReceived from Godot:")
    print(json.dumps(message, indent=4))

    # --- Example: extract state from message (adapt to your actual JSON) ---
    arena = message.get("arena", {})
    tank = message.get("tank", {})
    goal = message.get("goal", {})

    # Example of preprocessing (normalize positions)
    arena_w = arena.get("width", 1.0)
    arena_h = arena.get("height", 1.0)

    tank_x = tank.get("x", 0.0) / arena_w
    tank_y = tank.get("y", 0.0) / arena_h
    goal_x = goal.get("x", 0.0) / arena_w
    goal_y = goal.get("y", 0.0) / arena_h

    state_vector = [tank_x, tank_y, goal_x, goal_y]

    # ----------------------------------------------------------------------
    # 🔥 TODO: YOUR BACKPROP / TRAINING HERE
    # This is where you'd do something like (pseudo-code):
    #
    #   state_tensor = torch.tensor(state_vector).float().unsqueeze(0)
    #   predicted_value = model(state_tensor)
    #   loss = loss_fn(predicted_value, target_value)
    #   optimizer.zero_grad()
    #   loss.backward()
    #   optimizer.step()
    #
    # And then compute an action, e.g.:
    #   action = policy(state_tensor)
    #
    # For now, we’ll just send a dummy action back.
    # ----------------------------------------------------------------------

    # Dummy action example: always "turn right" with some throttle
    action = {
        "turn": 1.0,      # e.g. -1 = left, 0 = straight, 1 = right
        "throttle": 0.5,  # 0..1
    }

    # You might also send back value estimates, etc.
    response = {
        "action": action,
        "state": state_vector,
        "info": {
            "message": "Dummy action from Python. Replace with your model output."
        }
    }

    print("\nSending back to Godot:")
    print(json.dumps(response, indent=4))

    send_json(sock, response)


def receive_loop() -> None:
    """
    Connects to the Godot TCP server and continuously:
      - receives JSON messages
      - calls on_message() for each one
      - sends back JSON responses
    """
    print(f"Connecting to {HOST}:{PORT} ...")
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.connect((HOST, PORT))
        print("Connected! Waiting for JSON messages...\n")

        buffer = ""
        decoder = json.JSONDecoder()

        while True:
            chunk = sock.recv(4096)
            if not chunk:
                print("Connection closed by server.")
                break

            buffer += chunk.decode("utf-8")

            # Try to parse as many JSON objects as possible
            while buffer:
                buffer = buffer.lstrip()
                if not buffer:
                    break

                try:
                    obj, idx = decoder.raw_decode(buffer)
                except json.JSONDecodeError:
                    # Not enough data yet to parse a full JSON object
                    break

                raw_json = buffer[:idx]
                buffer = buffer[idx:]

                # obj is a dict parsed from Godot
                try:
                    on_message(sock, obj)
                except Exception as e:
                    print(f"Error in on_message: {e}")


if __name__ == "__main__":
    receive_loop()

Connecting to 127.0.0.1:5000 ...
Connected! Waiting for JSON messages...

Raw JSON from Godot:
{"arena":{"height":1080.0,"width":1920.0},"goal":{"x":1463.0,"y":460.0},"tank":{"x":600.368225097656,"y":539.806823730469}}

Parsed JSON from Godot:
{
    "arena": {
        "height": 1080.0,
        "width": 1920.0
    },
    "goal": {
        "x": 1463.0,
        "y": 460.0
    },
    "tank": {
        "x": 600.368225097656,
        "y": 539.806823730469
    }
}
----------------------------------------
Raw JSON from Godot:
{"arena":{"height":1080.0,"width":1920.0},"goal":{"x":1463.0,"y":460.0},"tank":{"x":517.884155273438,"y":578.267395019531}}

Parsed JSON from Godot:
{
    "arena": {
        "height": 1080.0,
        "width": 1920.0
    },
    "goal": {
        "x": 1463.0,
        "y": 460.0
    },
    "tank": {
        "x": 517.884155273438,
        "y": 578.267395019531
    }
}
----------------------------------------
Raw JSON from Godot:
{"arena":{"height":1080.0,"width":1920.0},"goal

KeyboardInterrupt: 